In [62]:
from __future__ import print_function
import numpy as np
import pandas as pd
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
import lime
import lime.lime_tabular
from int_met import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality',discretize=False, balance=True, dataset_folder=dataset_folder)

In [3]:
explainer = lime.lime_tabular.LimeTabularExplainer(dataset.train, feature_names=dataset.feature_names, class_names=dataset.class_target,discretize_continuous=False,)

In [4]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=70, n_jobs=5, random_state=1)
c.fit(dataset.train, dataset.labels_train)
predict_fn = lambda x: c.predict(x)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))

Train 1.0
Test 0.8504784688995215


In [5]:
import pickle
pickling_on = open("rfmort.pickle","wb")
pickle.dump(c, pickling_on)
pickling_on.close()

In [15]:
exp_fn = lambda i: explainer.explain_instance(dataset.test[i], c.predict_proba, num_features=5, top_labels=1)
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in range(len(xtest)):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [148]:
dataset.categorical_names

{1: ['Black', 'Other', 'White'],
 2: ['Female', 'Male'],
 6: ['Abnormal Test',
  'Arrhythmia',
  'Chest Pain',
  'Conduction System Disease',
  'Dizzy, Fatigue',
  'Known CAD',
  'Other',
  'Palpitation',
  'Pre-Operation',
  'Risk Factor',
  'Rule out Ischemia',
  'Screening, Research',
  'Shortness of Breath'],
 7: ['No', 'Yes'],
 8: ['No', 'Yes'],
 9: ['No', 'Yes'],
 10: ['No', 'Yes'],
 11: ['No', 'Yes'],
 12: ['No', 'Yes'],
 13: ['No', 'Yes']}

In [149]:
import copy
test_ori = []
enc_feats = []
sz = dataset.test.shape
for i in range(sz[0]):
    temp = list(copy.deepcopy(dataset.test[i]))
    for j in range(sz[0]):
        if j in dataset.categorical_features:
            temp[j] = dataset.categorical_names[j][int(temp[j])].replace(',','')
    test_ori.append(temp)

In [150]:
ori_data = pd.DataFrame(test_ori, columns=dataset.feature_names)

In [151]:
feats = copy.deepcopy(dataset.feature_names)

In [152]:
a = list(np.array(feats)[order])

In [162]:
view1 = ori_data.reindex(columns=a)
viewno = copy.deepcopy(ori_data)

In [ ]:
view1

In [163]:
#view1.sort_values(by='age', inplace=True)

In [164]:
view1.insert(loc=0,column='dead',value=dataset.labels_test)
viewno.insert(loc=0,column='dead',value=dataset.labels_test)

In [166]:
yesmap={0:'No', 1:'Yes'}
yes_no = lambda x: yesmap[x] 

In [169]:
view1['dead']=view1['dead'].apply(yes_no)
viewno['dead']=viewno['dead'].apply(yes_no)

In [170]:
view1.to_csv('amortlime.csv',index=False)
viewno.to_csv('amortno.csv',index=False)

In [25]:
exp1 = exp_fn_blk(dataset.test, exp_fn)

In [103]:
num_feats = len(dataset.feature_names)
sz = exp1.shape
imp = np.zeros(num_feats)
for i in range(sz[0]):
    for j in range(sz[1]):
        imp[int(exp1[i][j][0])] = imp[int(exp1[i][j][0])] + sz[1] - j
    

In [171]:
imp

array([4178,    0, 1481, 3341,    0, 2503,    0,    0,    0,    0,    0,
          0, 1037,    0])

In [ ]:
feats

In [107]:
imp = imp.astype(int)

In [115]:
order = np.argsort(-imp)